### Data Copnversion From Labelme to Coco Format

In [ ]:
import os
import json
import glob
from PIL import Image
from tqdm import tqdm

def labelme_shape_to_coco_annotation(shape, image_id, annotation_id, label_to_id):
    points = shape['points']
    label = shape['label']

    # Flatten the list of points for segmentation
    segmentation = [list(sum(points, []))]

    # Calculate bounding box
    xs = [pt[0] for pt in points]
    ys = [pt[1] for pt in points]
    x_min, y_min = min(xs), min(ys)
    width, height = max(xs) - x_min, max(ys) - y_min
    bbox = [x_min, y_min, width, height]

    area = width * height

    annotation = {
        'id': annotation_id,
        'image_id': image_id,
        'category_id': label_to_id[label],
        'segmentation': segmentation,
        'bbox': bbox,
        'iscrowd': 0,
        'area': area
    }

    return annotation


def convert_labelme_to_coco(labelme_folder, output_json_path, image_folder=None):
    label_files = glob.glob(os.path.join(labelme_folder, '*.json'))

    images = []
    annotations = []
    categories = []
    label_to_id = {}

    image_id = 1
    annotation_id = 1
    category_id = 1

    for label_file in tqdm(label_files, desc="Processing Labelme JSONs"):
        with open(label_file, 'r') as f:
            data = json.load(f)

        # Image filename
        filename = data['imagePath']
        image_path = os.path.join(image_folder or labelme_folder, filename)

        if not os.path.exists(image_path):
            print(f" Image file not found: {image_path}, skipping.")
            continue

        # Get image dimensions
        try:
            with Image.open(image_path) as img:
                width, height = img.size
        except Exception as e:
            print(f" Failed to open image {image_path}: {e}")
            continue

        # Add image entry
        images.append({
            'id': image_id,
            'file_name': filename,
            'width': width,
            'height': height
        })

        # Process shapes
        for shape in data.get('shapes', []):
            label = shape['label']
            if label not in label_to_id:
                label_to_id[label] = category_id
                categories.append({
                    'id': category_id,
                    'name': label,
                    'supercategory': 'none'
                })
                category_id += 1

            annotation = labelme_shape_to_coco_annotation(shape, image_id, annotation_id, label_to_id)
            annotations.append(annotation)
            annotation_id += 1

        image_id += 1

    coco_output = {
        'images': images,
        'annotations': annotations,
        'categories': categories
    }

    os.makedirs(os.path.dirname(output_json_path), exist_ok=True)
    with open(output_json_path, 'w') as f:
        json.dump(coco_output, f, indent=4)

    print(f"\n COCO dataset saved to: {output_json_path}")
    print(f" Total images: {len(images)}, Annotations: {len(annotations)}, Categories: {len(categories)}")



if __name__ == '__main__':
    labelme_folder = 'data/labelme' # Folder with Labelme .json files
    image_folder = 'data/images'          # Folder with image files (set to None if same as JSONs)
    output_json_path = 'data/coco.json'   # Output COCO annotation path

    convert_labelme_to_coco(labelme_folder, output_json_path, image_folder)
